In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt
import gym
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
n_inputs = 4
n_hidden = 4
n_outputs = 1
initializer = tf.contrib.layers.variance_scaling_initializer()

learning_rate = .01

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden  = fully_connected(X, n_hidden, activation_fn=tf.nn.elu,
                         weights_initializer=initializer)
logits  = fully_connected(hidden, n_outputs, activation_fn=None,
                         weights_initializer=initializer)
print(logits)
outputs = tf.nn.sigmoid(logits)
print(outputs)

p_left_and_right = tf.concat(axis=1, values=[outputs, 1-outputs])
action = tf.multinomial(tf.log(p_left_and_right), num_samples=1)

Tensor("fully_connected_1/BiasAdd:0", shape=(?, 1), dtype=float32)
Tensor("Sigmoid:0", shape=(?, 1), dtype=float32)


In [3]:
y = 1. - tf.to_float(action)
learning_rate = 1.0
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]
print(gradients)
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, \
                              shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))

training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

[<tf.Tensor 'gradients/fully_connected/MatMul_grad/tuple/control_dependency_1:0' shape=(4, 4) dtype=float32>, <tf.Tensor 'gradients/fully_connected/BiasAdd_grad/tuple/control_dependency_1:0' shape=(4,) dtype=float32>, <tf.Tensor 'gradients/fully_connected_1/MatMul_grad/tuple/control_dependency_1:0' shape=(4, 1) dtype=float32>, <tf.Tensor 'gradients/fully_connected_1/BiasAdd_grad/tuple/control_dependency_1:0' shape=(1,) dtype=float32>]


In [4]:
def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.empty(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + \
            discount_rate * cumulative_rewards
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate)\
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(rewards-reward_mean)/reward_std for rewards in all_discounted_rewards]

In [5]:
discount_rewards([10, 0, -50], discount_rate=.8)
discount_and_normalize_rewards([[10, 0, -50], [10, 20]], .8)

[array([-0.28435071, -0.86597718, -1.18910299]),
 array([1.26665318, 1.0727777 ])]

In [24]:
n_iterations = 250
n_max_steps = 1000
n_games_per_update = 10
save_iterations = 10
discount_rate = .95
env = gym.make('CartPole-v0')
with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        all_rewards = []
        all_gradients = []
        for game in range(n_games_per_update):
            current_rewards = []
            current_gradients = []
            obs = env.reset()
            for step in range(n_max_steps):
                action_val, gradients_val = sess.run(\
                        [action, gradients],
                        feed_dict={X:obs.reshape(1, -1)})
                obs, reward, done, info = \
                    env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)
        # At this point we have run the policy for 
        # n_gmaes_per_update episoedes, and are ready
        # for a policy update
        all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate)
        feed_dict = {}
        tweaked_grads = []
        for var_index, grad_placehoder in enumerate(gradient_placeholders):
            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
                            for game_index, rewards in enumerate(all_rewards)
                            for step, reward in enumerate(rewards)], axis=0)
            feed_dict[grad_placehoder] = mean_gradients
        sess.run(training_op, feed_dict=feed_dict)
        if iteration%save_iterations==0:
            saver.save(sess, './my_policy_net_pg.ckpt')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


KeyboardInterrupt: 